# IBM Data Science Specialization Capstone Project

This Jupyter Notebook will be mainly used to complete and submit the capstone project for the IBM Data Science specialization on Coursera.

## Part 1

We will first start by importing the required libraries.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

We will retrieve our data for Toronto postal codes from Wikipedia's page. The URL for the page is stored in the variable <code>url</code>, then using BeautifulSoup we store the table on that page into the variable <code>table</code>.

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text
soup = BeautifulSoup(data, 'html5lib')
table = soup.find('table')

We set up a pandas dataframe and call it <code>toronto_pc</code> consisting of three columns: PostalCode, Borough and Neighborhood. We then go through the text in the saved table to extract the relevant data and store it in our dataframe.

In [3]:
columns = ['PostalCode', 'Borough', 'Neighborhood']

toronto_pc = pd.DataFrame(columns=columns)

for item in table.get_text().split('\n'):
    if item != '':
        code = item[0:3]
        assignment = item[3:].strip()
        if assignment != 'Not assigned':
            boroughs = assignment.split('(')[0].strip()
            neighborhoods = assignment.split('(')[1].replace(')', ' ').replace(' /', ',').strip()
            toronto_pc = toronto_pc.append({'PostalCode':code, 'Borough':boroughs, 'Neighborhood':neighborhoods}, ignore_index=True)

toronto_pc['Borough'] = toronto_pc['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Finally, we use the <code><b>.shape</b></code> method to print the number of rows in our dataframe.

In [4]:
print('There are {} rows in our dataframe.'.format(toronto_pc.shape[0]))

There are 103 rows in our dataframe.


## Part 2

We will start by getting the geospatial data from the course's CSV file (since the other method was returning <code>None</code> most of the time).
We will directly store the data of the CSV file in a pandas dataframe called <code>geodata</code>.

In [5]:
geodata = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

Let's take a look at the first five lines of the new dataframe.

In [6]:
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now let's check how many rows there are in the new dataframe just to make sure that they are the same as the number of postal codes that we got from Part 1.

In [7]:
print('There are {} rows in the geospatial data dataframe.'.format(geodata.shape[0]))

There are 103 rows in the geospatial data dataframe.


Now let us merge the dataframe that we got in Part 1 with the latitudes and longitudes that we got from the geospatial dataframe, and let's see the result.

In [8]:
toronto_geo = toronto_pc.join(geodata.set_index('Postal Code'), on='PostalCode')
toronto_geo

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Part 3

The following cell should only be run once and only if the Folium and Geopy libraries are not installed.

In [9]:
!pip install folium
!pip install geopy

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 5.3 MB/s  eta 0:00:01
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


Let's start by importing the required libraries for this part.

In [10]:
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
import numpy as np

We will first get the coordinates for Toronto so we can later use them to center the map on the city.

In [11]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

Then we will use Folium to show the map of Toronto and mark on it the locations of the postal codes that we got in Part 2.

In [12]:
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(toronto_geo['Latitude'], toronto_geo['Longitude'], toronto_geo['Borough'], toronto_geo['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_toronto)

map_toronto

Now we will start utilizing Foursquare to explore the different areas of Toronto.

In [13]:
# The code was removed by Watson Studio for sharing.

We will define a function that gets all nearby venues within a certain radius of a certain set of coordinates.

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PC Latitude', 
                  'PC Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Let's pull the venues within 500 meters of the coordinates listed in the <code>toronto_geo</code> dataframe and store them in a new dataframe called <code>toronto_venues</code>.

In [15]:
toronto_venues = getNearbyVenues(names=toronto_geo['PostalCode'], latitudes=toronto_geo['Latitude'], longitudes=toronto_geo['Longitude'], radius=500)

Let's check the size of the new dataframe and the first five rows.

In [16]:
print(toronto_venues.shape)
toronto_venues.head()

(2120, 7)


,PostalCode,PC Latitude,PC Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Let's see how many venues were found in each area.

In [17]:
toronto_venues.groupby('PostalCode').count()

,PC Latitude,PC Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M1B,2,2,2,2,2,2
M1C,1,1,1,1,1,1
M1E,9,9,9,9,9,9
M1G,3,3,3,3,3,3
M1H,9,9,9,9,9,9
...,...,...,...,...,...,...
M9N,2,2,2,2,2,2
M9P,8,8,8,8,8,8
M9R,4,4,4,4,4,4


There seems to be some areas that do not have any venues attached to them.

In [18]:
missing_pc = toronto_geo[~toronto_geo['PostalCode'].isin(toronto_venues['PostalCode'])]
missing_pc

,PostalCode,Borough,Neighborhood,Latitude,Longitude
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
45,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714
95,M1X,Scarborough,Upper Rouge,43.836125,-79.205636


We will loop through the missing locations and add venues named <b>'None'</b> to serve as placeholders. The coordinates for these placeholder venues will be the same as the ones of the PostalCode.

In [19]:
for pcode in missing_pc['PostalCode']:
    pc_lat = missing_pc[missing_pc['PostalCode'] == pcode]['Latitude']
    pc_lng = missing_pc[missing_pc['PostalCode'] == pcode]['Longitude']
    ven = 'None'
    ven_lat = pc_lat
    ven_lng = pc_lng
    ven_cat = 'None'
    toronto_venues = toronto_venues.append({'PostalCode': pcode, 'PC Latitude': pc_lat, 'PC Longitude': pc_lng, 'Venue': ven, 'Venue Latitude': ven_lat, 'Venue Longitude': ven_lng, 'Venue Category': ven_cat}, ignore_index=True)

In [20]:
toronto_venues.groupby('PostalCode').count()

,PC Latitude,PC Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M1B,2,2,2,2,2,2
M1C,1,1,1,1,1,1
M1E,9,9,9,9,9,9
M1G,3,3,3,3,3,3
M1H,9,9,9,9,9,9
...,...,...,...,...,...,...
M9N,2,2,2,2,2,2
M9P,8,8,8,8,8,8
M9R,4,4,4,4,4,4


Now we have the correct number of rows in our grouped dataframe.

#### How many unique venue categories are there?

In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 273 uniques categories.


Let's use One-Hot Encoding to create a separate column for each distinct venue category. This will change our categorical data into numerical data. We will also add the Postal Code column to the beginning of the dataframe so we can group the data later.

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postal code column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move postal code column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The size of the One-Hot encoded data is:

In [23]:
toronto_onehot.shape

(2124, 274)

Now let's group the above dataset by postal code and by using the <code>mean()</code> function.

In [24]:
toronto_grouped = toronto_onehot.groupby("PostalCode").mean().reset_index()
toronto_grouped

,PostalCode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M9N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,M9P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,M9R,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,M9V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


The size of the grouped data is:

In [25]:
toronto_grouped.shape

(103, 274)

We will define a function that returns the most common venues near a certain location.

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

We will use the data that we got from the grouped dataframe to extract the 10 most common venues at each postal code and store them in a new dataframe called <code>pc_venues_sorted</code>.

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pc_venues_sorted = pd.DataFrame(columns=columns)
pc_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    pc_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

pc_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Print Shop,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
1,M1C,Bar,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Farmers Market
2,M1E,Rental Car Location,Medical Center,Intersection,Bank,Mexican Restaurant,Donut Shop,Restaurant,Breakfast Spot,Electronics Store,Drugstore
3,M1G,Coffee Shop,Korean BBQ Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,M1H,Hakka Restaurant,Bank,Lounge,Athletics & Sports,Bakery,Caribbean Restaurant,Thai Restaurant,Gas Station,Fried Chicken Joint,Donut Shop


### K-Means Clustering

We will now proceed with clustering our locations based on common venues using K-Means Clustering. We will cluster our data into 5 clusters. To do that, we will create a separate dataframe to be used to fit the clustering data which is similar to the <code>toronto_grouped</code> dataframe but without the <code>PostalCode</code> column.

In [28]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

kmeans = KMeans(n_clusters=kclusters, random_state = 4).fit(toronto_grouped_clustering)

<b><u>Note:</u> the reason why we set a <code>random_state</code> value for KMeans in the above cell is so we can get a consistant result. Otherwise, the results will differ every time the notebook is run and the analysis in the markdown cells below will be incorrect.</b>

Let's check the labels for the first 10 rows.

In [29]:
kmeans.labels_[0:10]

array([2, 0, 2, 2, 2, 2, 2, 2, 2, 0], dtype=int32)

We'll add the labels as a new column to the sorted venues dataframe then merge the dataframe with the Toronto geodata dataframe to get a new <code>toronto_merged</code> dataframe.

In [30]:
pc_venues_sorted.insert(0, 'Cluster labels', kmeans.labels_)

toronto_merged = toronto_geo

toronto_merged = toronto_merged.join(pc_venues_sorted.set_index('PostalCode'), on='PostalCode')

Let's view the first 5 lines of the new dataframe.

In [31]:
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Food & Drink Shop,Park,Fast Food Restaurant,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Hockey Arena,Portuguese Restaurant,Pizza Place,Coffee Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Theater,Breakfast Spot,Pub,Café,Hotel,Spa,Beer Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2,Clothing Store,Accessories Store,Coffee Shop,Boutique,Miscellaneous Shop,Gift Shop,Furniture / Home Store,Vietnamese Restaurant,Construction & Landscaping,Dim Sum Restaurant
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Yoga Studio,Beer Bar,Smoothie Shop,Sandwich Place,Salad Place,Burger Joint,Burrito Place,Restaurant


Let's plot the clusters.

In [32]:
map_clusters = folium.Map(locations=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

#### Cluster 1

In [33]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,North York,0,Café,Gym,Japanese Restaurant,Caribbean Restaurant,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
12,Scarborough,0,Bar,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Farmers Market
34,North York,0,Metro Station,Furniture / Home Store,Massage Studio,Coffee Shop,Caribbean Restaurant,Miscellaneous Shop,Bar,Doner Restaurant,Distribution Center,Dog Run
39,North York,0,Café,Japanese Restaurant,Bank,Chinese Restaurant,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
57,North York,0,Baseball Field,Furniture / Home Store,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
58,Scarborough,0,College Stadium,General Entertainment,Skating Rink,Café,Comic Shop,Dim Sum Restaurant,Ethiopian Restaurant,Colombian Restaurant,Escape Room,Electronics Store
63,York,0,Bus Line,Caribbean Restaurant,Convenience Store,Breakfast Spot,Yoga Studio,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant
94,Etobicoke Northwest,0,Rental Car Location,Bar,Garden Center,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


Cluster 1 has a variety of locations as the most common venue.

#### Cluster 2

In [34]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Food & Drink Shop,Park,Fast Food Restaurant,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
21,York,1,Park,Women's Store,Pool,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Department Store
35,East York/East Toronto,1,Intersection,Park,Convenience Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
40,North York,1,Park,Bus Stop,Airport,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
61,Central Toronto,1,Park,Swim School,Bus Line,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dessert Shop
64,York,1,Park,Convenience Store,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
66,North York,1,Park,Convenience Store,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
68,Central Toronto,1,Trail,Park,Sushi Restaurant,Bus Line,Jewelry Store,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run
77,Etobicoke,1,Park,Mobile Phone Shop,Bus Line,Sandwich Place,Dog Run,Diner,Discount Store,Distribution Center,Doner Restaurant,Dessert Shop
85,Scarborough,1,Park,Intersection,Playground,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop


Parks seem to be the most common venue in Cluster 2.

#### Cluster 3

In [35]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2,Hockey Arena,Portuguese Restaurant,Pizza Place,Coffee Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,Downtown Toronto,2,Coffee Shop,Park,Bakery,Theater,Breakfast Spot,Pub,Café,Hotel,Spa,Beer Store
3,North York,2,Clothing Store,Accessories Store,Coffee Shop,Boutique,Miscellaneous Shop,Gift Shop,Furniture / Home Store,Vietnamese Restaurant,Construction & Landscaping,Dim Sum Restaurant
4,Queen's Park,2,Coffee Shop,Sushi Restaurant,Yoga Studio,Beer Bar,Smoothie Shop,Sandwich Place,Salad Place,Burger Joint,Burrito Place,Restaurant
6,Scarborough,2,Fast Food Restaurant,Print Shop,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,2,Coffee Shop,Café,Hotel,Restaurant,Gym,Japanese Restaurant,Deli / Bodega,Steakhouse,American Restaurant,Asian Restaurant
99,Downtown Toronto,2,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Men's Store,Mediterranean Restaurant,Yoga Studio,Hotel,Fast Food Restaurant
100,East Toronto Business,2,Yoga Studio,Garden Center,Smoke Shop,Light Rail Station,Brewery,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Auto Workshop
101,Etobicoke,2,Home Service,Business Service,Baseball Field,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio


Cluster 3 seems to be the locations where the most common venue are coffee shops.

#### Cluster 4

In [36]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,3,None,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dessert Shop
11,Etobicoke,3,None,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dessert Shop
45,North York,3,None,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dessert Shop
95,Scarborough,3,None,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dessert Shop


Cluster 4 is the cluster of locations that did not have venues and that we added previously.

#### Cluster 5

In [37]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,North York,4,Gym,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
83,Central Toronto,4,Trail,Gym,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


Cluster 5 is where gyms are the most common venues.

---
This concludes the requirements of Week 3's assignment for the Capstone Project Course.